In [ ]:
%load_ext autoreload
%autoreload 2
#%pip install trueskill
import sys
print(sys.version)
import networkx as nx
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import time

#from ..solvers import exact
sys.path.append("..")
sys.path.append('../solvers')

from helper_functions import draw_graph_with_FAS
from helper_functions import introduce_cycles, generate_random_DAG, gen_graph_DAG, run

#import solvers
from solvers import exact, italjani
from solvers import remove_cycle_edges_by_dfs
from solvers import remove_cycle_edges_by_minimum_feedback_arc_set_greedy
from solvers import remove_cycle_edges_by_minimum_feedback_arc_set_greedy_parallel
from solvers import remove_cycle_edges_by_hierarchy
#from solvers import exact

In [ ]:
df_graphs = pd.read_csv('../components.csv')
for line in df_graphs.iterrows():
    with open('../' + line[1][3], 'rb') as f:
        graph = pickle.load(f)
        if graph.number_of_nodes() > 5 and graph.number_of_nodes() < 20:
            print(line[1][3])
            print(graph.number_of_nodes())
            break
#nx.draw(graph)
#plt.show()

In [ ]:
file_name = '../pickles/smallworld-0017-80-10-24.al_0.pickle'
with open(file_name, 'rb') as f:
    graph = pickle.load(f)
    print(graph.number_of_nodes())
    print(graph.number_of_edges())
    nx.draw(graph, with_labels=True)
    plt.show()
    #print(graph.edges())
    #print(graph.nodes

In [ ]:
with open(file_name, 'rb') as f:
    graph = pickle.load(f)
    rez_exact = exact.exact_with_wcnf(graph)
    

In [ ]:
print(rez_exact)
print(len(rez_exact))

### Color the solution

In [ ]:
with open(file_name, 'rb') as f:
    graph = pickle.load(f)
    draw_graph_with_FAS(graph, rez_exact)
    '''print(graph.number_of_nodes())
    print(graph.number_of_edges())
    edge_colors = []

    for u, v in graph.edges():
        #print(u,v)
        if (u,v) in rez_exact:
            print(u,v)
            edge_colors.append('r')
        else:
            edge_colors.append('b')
    nx.draw(graph, edge_color=edge_colors, with_labels=True)
    plt.show()'''

## Heuristics

In [ ]:
with open(file_name, 'rb') as f:
    graph = pickle.load(f)
    rez_i = italjani.get_some_FAS(graph)
    draw_graph_with_FAS(graph, rez_i)
    
    graph2 = graph.copy()
    for u,v in rez_i:
        graph2.remove_edge(u,v)
    print(nx.is_directed_acyclic_graph(graph2))
    print(nx.is_directed_acyclic_graph(graph))

In [ ]:
print(rez_i)
print(len(rez_i))

In [ ]:
with open(file_name, 'rb') as f:
    graph = pickle.load(f)
    rez_d = remove_cycle_edges_by_dfs.remove_edges_dfs(graph)

In [ ]:
print(rez_d)
print(len(rez_d))

In [ ]:
with open(file_name, 'rb') as f:
    graph = pickle.load(f)
    rez_g = remove_cycle_edges_by_minimum_feedback_arc_set_greedy.remove_edges_FAS_greedy(graph)

In [ ]:
print(rez_g)
print(len(rez_g))

In [ ]:
with open(file_name, 'rb') as f:
    graph = pickle.load(f)
    rez_gp = remove_cycle_edges_by_minimum_feedback_arc_set_greedy_parallel.remove_edges_FAS_greedy_parallel(graph)

In [ ]:
print(rez_gp)
print(len(rez_gp))

In [ ]:
with open(file_name, 'rb') as f:
    graph = pickle.load(f)
    rez_page = remove_cycle_edges_by_hierarchy.remove_edges_hierarchy(graph, "pagerank", "voting")

In [ ]:
print(rez_page)
print(len(rez_page))

In [ ]:
with open(file_name, 'rb') as f:
    graph = pickle.load(f)
    rez_ts = remove_cycle_edges_by_hierarchy.remove_edges_hierarchy(graph, "trueskill", "voting")

In [ ]:
print(rez_ts)
print(len(rez_ts))
draw_graph_with_FAS(graph, rez_ts)

## Generiranje grafov

In [ ]:
G, cycles = gen_graph_DAG(20, c=4)
#print(nx.is_directed_acyclic_graph(G))
#print(cycles)
e = [(cycle[i], cycle[(i+1) % len(cycle)]) for cycle in cycles for i in range(len(cycle))]

G2 = G.copy()
for u,v in e:
    if G2.has_edge(u,v):
        G2.remove_edge(u,v)
print(nx.is_directed_acyclic_graph(G2))

rez_exact = exact.exact_with_wcnf(G, cycles)
rez_exact = list(map(lambda x: (int(x[0]), int(x[1])), rez_exact))

#draw_graph_with_FAS(G, e)
G3 = G.copy()
for u,v in rez_exact:
    G3.remove_edge(u,v)
print(nx.is_directed_acyclic_graph(G3))
#print(G.edges())
#print(rez_exact)

draw_graph_with_FAS(G, rez_exact)

In [ ]:
for i in range(10):
    start = time.time()
    G, cycles = gen_graph_DAG(30, c=6)
    e = [(cycle[i], cycle[(i+1) % len(cycle)]) for cycle in cycles for i in range(len(cycle))]
    
    G2 = G.copy()
    for u,v in e:
        if (u,v) in G2.edges():
            G2.remove_edge(u,v)
    rez = nx.is_directed_acyclic_graph(G2)
    assert rez
    
    rez_exact = exact.exact_with_wcnf(G)
    rez_exact = list(map(lambda x: (int(x[0]), int(x[1])), rez_exact))

    rez_cycles = exact.exact_with_wcnf(G, cycles)
    rez_cycles = list(map(lambda x: (int(x[0]), int(x[1])), rez_cycles))
    #print(len(list(rez_exact)), len(list(rez_exact2)))
    G3 = G.copy()
    for u,v in rez_cycles:
        G3.remove_edge(u,v)
    r = nx.is_directed_acyclic_graph(G3)
    rez_cycles_d = exact.exact_with_wcnf(G3)
    rez_cycles_d = list(map(lambda x: (int(x[0]), int(x[1])), rez_cycles_d))
    G4 = G3.copy()
    for u,v in rez_cycles_d:
        G4.remove_edge(u,v)
    rez = nx.is_directed_acyclic_graph(G4)

    #print(time.time() - start)
    if not r:
        print(rez)
        print(len(rez_exact), len(rez_cycles), len(rez_cycles_d))
        print(len(rez_exact), len(rez_exact2))
        print(cycles)
        print(list(nx.simple_cycles(G3)))
        draw_graph_with_FAS(G, rez_exact)
        break

In [ ]:
G, cycles = gen_graph_DAG(30, c=6)
#edges = [(cycle[i], cycle[(i+1) % len(cycle)]) for cycle in cycles for i in range(len(cycle))]
cycles_edges = [[(cycle[i], cycle[(i+1) % len(cycle)]) for i in range(len(cycle))] for cycle in cycles]
edges = {}

for cycle_edges in cycles_edges:
    for edge in cycle_edges:
        #print(edge)
        if edge in edges:
            edges[edge] += 1
        else:
            edges[edge] = 1

print(edges)
removed = []
while len(edges) != 0:
    edges, cycles_edges, edge = run(edges, cycles_edges)
    removed.append(edge)
    
G2 = G.copy()
for u,v in removed:
    if (u,v) in G2.edges():
        G2.remove_edge(u,v)
rez = nx.is_directed_acyclic_graph(G2)
print(rez)
    
rez_exact = exact.exact_with_wcnf(G)
rez_exact = list(map(lambda x: (int(x[0]), int(x[1])), rez_exact))